## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bethel,US,41.3712,-73.4140,22.50,56,0,3.44,clear sky
1,1,Jamestown,US,42.0970,-79.2353,18.72,60,100,0.00,overcast clouds
2,2,Rikitea,PF,-23.1203,-134.9692,76.86,70,100,5.06,overcast clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,50.11,57,75,26.46,broken clouds
4,4,Terme,TR,41.2092,36.9739,46.33,73,94,4.27,light rain


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,76.86,70,100,5.06,overcast clouds
9,9,Kapaa,US,22.0752,-159.3190,80.47,63,0,1.99,clear sky
11,11,Butaritari,KI,3.0707,172.7902,82.11,76,22,20.94,few clouds
15,15,Hilo,US,19.7297,-155.0900,77.11,62,20,8.05,few clouds
31,31,Bengkulu,ID,-3.8004,102.2655,75.49,89,99,4.18,moderate rain
36,36,Atuona,PF,-9.8000,-139.0333,78.19,77,12,14.38,light rain
39,39,Saint-Philippe,RE,-21.3585,55.7679,78.39,83,43,10.13,scattered clouds
40,40,Karratha,AU,-20.7377,116.8463,83.79,60,69,17.11,broken clouds
50,50,Georgetown,MY,5.4112,100.3354,79.90,89,20,4.61,few clouds
53,53,Kedougou,SN,12.5579,-12.1743,82.94,14,17,3.60,few clouds


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,76.86,70,100,5.06,overcast clouds
9,9,Kapaa,US,22.0752,-159.3190,80.47,63,0,1.99,clear sky
11,11,Butaritari,KI,3.0707,172.7902,82.11,76,22,20.94,few clouds
15,15,Hilo,US,19.7297,-155.0900,77.11,62,20,8.05,few clouds
31,31,Bengkulu,ID,-3.8004,102.2655,75.49,89,99,4.18,moderate rain
36,36,Atuona,PF,-9.8000,-139.0333,78.19,77,12,14.38,light rain
39,39,Saint-Philippe,RE,-21.3585,55.7679,78.39,83,43,10.13,scattered clouds
40,40,Karratha,AU,-20.7377,116.8463,83.79,60,69,17.11,broken clouds
50,50,Georgetown,MY,5.4112,100.3354,79.90,89,20,4.61,few clouds
53,53,Kedougou,SN,12.5579,-12.1743,82.94,14,17,3.60,few clouds


In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,76.86,overcast clouds,-23.1203,-134.9692,
9,Kapaa,US,80.47,clear sky,22.0752,-159.3190,
11,Butaritari,KI,82.11,few clouds,3.0707,172.7902,
15,Hilo,US,77.11,few clouds,19.7297,-155.0900,
31,Bengkulu,ID,75.49,moderate rain,-3.8004,102.2655,
36,Atuona,PF,78.19,light rain,-9.8000,-139.0333,
39,Saint-Philippe,RE,78.39,scattered clouds,-21.3585,55.7679,
40,Karratha,AU,83.79,broken clouds,-20.7377,116.8463,
50,Georgetown,MY,79.90,few clouds,5.4112,100.3354,
53,Kedougou,SN,82.94,few clouds,12.5579,-12.1743,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping")
        

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,76.86,overcast clouds,-23.1203,-134.9692,People ThankYou
9,Kapaa,US,80.47,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
11,Butaritari,KI,82.11,few clouds,3.0707,172.7902,Isles Sunset Lodge
15,Hilo,US,77.11,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
31,Bengkulu,ID,75.49,moderate rain,-3.8004,102.2655,Grage Hotel Bengkulu
36,Atuona,PF,78.19,light rain,-9.8000,-139.0333,Villa Enata
39,Saint-Philippe,RE,78.39,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
40,Karratha,AU,83.79,broken clouds,-20.7377,116.8463,ibis Styles Karratha
50,Georgetown,MY,79.90,few clouds,5.4112,100.3354,Cititel Penang
53,Kedougou,SN,82.94,few clouds,12.5579,-12.1743,Relais de Kédougou


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))